# S2W5D7: 这里的黎明静悄悄 —— 复盘与白板模拟

## 1 📂 工程归档 (Code Refactoring)

首先，我们要像一个成熟的算法工程师一样，整理我们的军火库。
确保你的 `src/models/transformer_components.py` 文件中包含了以下所有积木（如果还没有，请今天花时间把它们整理进去）：

1.  **`PositionalEncoding`**: (Day 1)
2.  **`scaled_dot_product_attention`**: (Day 2)
3.  **`MultiHeadAttention`**: (Day 3)
4.  **`PositionwiseFeedForward`**: (Day 4)
5.  **`EncoderLayer`**: (Day 4)
6.  **`TransformerEncoder`** (可选): (Day 5)

**为什么要做这个？**
下周开始，当我们使用 Hugging Face 时，如果你想看懂源码，或者想魔改模型（比如把 Attention 换成由你发明的 Attention），你只需要打开这个文件，修改其中的几行即可。这就是**模块化**的力量。

## 2 🧠 白板模拟面试 (The Whiteboard Challenge)

现在，我将扮演**面试官**。请在你的 Notebook 中（或者找一张真实的白纸），尝试**不看代码、不看资料**，回答以下 4 个核心问题。

这 4 道题覆盖了 Transformer 面试考点的 **80%**。

### 🔥 Question 1: 请手画Transformer Encoder的内部结构图。
**考察点**: 你脑子里有没有清晰的数据流图。

**要求**:
* 画出 Input Embedding 和 Positional Encoding。
* 画出 N 个 Layer 的堆叠。
* **重点**: 在一个 Layer 内部，画出 `Multi-Head Attn` -> `Add & Norm` -> `FFN` -> `Add & Norm` 的连接顺序。
* **自我检查**: 你有没有画漏 **Residual Connection (那个绕过去的箭头)**？


### 🔥 Question 2: 为什么 Self-Attention 需要除以 $\sqrt{d_k}$ (Scale)？
**考察点**：对数学细节的理解

**参考答案**：因为当 $d_k$ 增大时，向量点积的结果 ($Q \cdot K^T$) 的方差会随之增大，导致数值在数量级上变得很大。这些大数值经过 Softmax 函数时，会落入激活函数的饱和区（梯度几乎为 0 的区域），从而导致梯度消失，模型无法有效训练。除以 $\sqrt{d_k}$ 是一种缩放操作，目的是将点积结果的分布拉回到均值为 0、方差为 1 的范围内，使 Softmax 的梯度保持在敏感区域。

### 🔥 Question 3: 这里的 Mask 是干什么用的？Padding Mask 和 Sequence Mask 有什么区别？
**考察点**: 对 NLP 变长序列处理的理解。

**参考答案**：关于 Mask，在 Transformer 里主要解决两个问题：一个是长度对齐，一个是防止偷看。首先是 Padding Mask。因为我们在训练时，一个 Batch 里的句子长度是不一样的，短句子后面补了很多 0。在算 Attention 的时候，我不希望模型去关注这些没有意义的 0，所以会在 Softmax 之前，把这些位置设为负无穷，这样它们的权重就自动变成 0 了。这在 Encoder 和 Decoder 里都会用到。其次是 Sequence Mask（也叫 Look-ahead Mask）。这个主要用在 Decoder 里，比如像 GPT 这种生成任务。因为在预测第 $t$ 个词的时候，模型是绝对不能看到 $t+1$ 之后的内容的（不能偷看答案）。所以我们需要用一个上三角矩阵，把未来的信息给遮挡住，保证因果性。

### 🔥 Question 4: Multi-Head Attention 的计算量比 Single-Head 大吗？
**考察点**: 对维度的直觉

**参考答案**：理论计算量（FLOPs）和参数量是基本一致的。虽然头的数量增加了（比如 8 个头），但每个头的特征维度 $d_k$ 相应减小了（$d_{model} / 8$）。在矩阵乘法中，Attention(Q, K, V) 的计算复杂度主要与 $d_{model}$ 的平方相关。将一个大矩阵运算拆解为多个小矩阵的并行运算，总的乘法加法次数几乎没有变化。多头的设计主要是为了增加模型捕捉不同子空间特征的能力，而非增加计算负担。


## 3. 🛠️ 具身智能专项复盘 (Embodied AI Special)

作为未来的具身智能算法工程师，你需要多思考这一层：

**Q: 如果让你把这周写的 Transformer 用到机器人手臂控制上，输入和输出分别是什么？**

* **思考方向**:
    * **输入 ($X$)**: 不再是单词 ID，而是 **[关节角度, 关节速度, 目标位置]** 组成的连续向量序列（过去 10 帧的历史状态）。
    * **Embedding**: 不需要查表 (`nn.Embedding`)，而是直接用 `nn.Linear` 把上述状态向量映射到 $d_{model}$ 维度。
    * **位置编码**: 代表 **“时间步”** (t-10, t-9, ... t)。
    * **输出 ($Y$)**: 下一时刻机器人的 **[目标关节力矩]**。

这就是所谓的 **Action Transformer (ACT)** 或 **Decision Transformer** 的核心原理。你学会了 Transformer，就等于学会了控制机器人。

## 4. 📝 总结与休息

**Week 5 任务清单**:
- [x] Day 1: 搞懂位置编码 (PE)。
- [x] Day 2: 手撕 Scaled Dot-Product Attention (核心)。
- [x] Day 3: 实现 Multi-Head 机制。
- [x] Day 4: 组装 Encoder Layer (Add & Norm, FFN)。
- [x] Day 5: 搭建 BERT 原型 (Encoder Stack)。
- [x] Day 6: 联调 ViT (Patch Embedding)。
- [x] Day 7: 归档代码，通过面试自测。


### 📅 下周预告 (Week 6)

下周，我们将走出“造轮子”的实验室，进入“工业界”的战场。

* **工具**: **Hugging Face (`transformers` 库)**。
* **任务**: 加载预训练好的 **BERT-Chinese** 模型。
* **实战**: 找一个真实的中文数据集，微调 BERT，做一个**情感分类**或**意图识别**项目。
* **简历价值**: 这将是你简历上第一个“基于预训练模型的 NLP 落地项目”。

好好休息，周一我们满血复活，开始 **Hugging Face** 之旅！👋